# Derivates in Theano : 도함수 계산

## Computing Gradient 

파라미터 x에 대한 y의 식의 도함수를 계산하는 함수를 만들어 보자. 
이를 수행하기 위해서 T.grad 매크로를 사용한다 
예를 들어, x에 대해 $x^2$의 식이 있다면 $d(x^2)/dx = 2 \cdot x$ 이 된다.


In [2]:
import theano
import theano.tensor as T
from theano import pp

In [3]:
x = T.dscalar('x')
y = x ** 2
gy = T.grad(y,x)
pp(gy) # print out the gradient prior to optimization

'((fill((x ** TensorConstant{2}), TensorConstant{1.0}) * TensorConstant{2}) * (x ** (TensorConstant{2} - TensorConstant{1})))'

In [4]:
f = theano.function([x], gy)
f(4)

array(8.0)

In [5]:
f(94.2)

array(188.4)

이 예제에서 pp(gy)로 정확한 symbolic gradient를 계산해 볼 수 있다.
fill((x**2), 1.0) 은  x**2와 같은 shape으로 매트릭스를 만들고 1.0으로 채우라는 뜻이다. 

|Note|
|:---|
|optimizer 는 symbolic gradient 식으로 단순화됨. 컴파일된 function의 내부 속성을 공부하면 볼 수 있음.<br> 최적화 이후에 인풋의 더블이 되도록 그래프의 node를 왼쪽에 적용.|

In [8]:
pp(f.maker.fgraph.outputs[0])

'(TensorConstant{2.0} * x)'

위에 정의된 logistic function 같은 복잡한 표현식의 gradient도 계산 가능하다.
logistic의 도함수는 : $ds(x)/dx = s(x) \cdot (1 - s(x))$

In [9]:
x = T.dmatrix('x')
s = T.sum(1 / (1 + T.exp(-x)))
gs = T.grad(s, x)
dlogistic = theano.function([x], gs)
dlogistic([[0, 1], [-1, -2]])

array([[ 0.25      ,  0.19661193],
       [ 0.19661193,  0.10499359]])

보통 스칼라 표현식 s에서는 , T.grad(s, w) 는 $\frac{\partial s}{\partial w}$를 계산하기 위한 theano 표현식으로 쓰인다. 이 방법에서는  Theano는 많은 입력을 가지는 함수의 경우에도  효율적인 symbolic differentiation을 수행할 수 있다 (as the expression returned by T.grad will be optimized during compilation).

|Note|
|:---|
| T.grad의 두번째 인자는 list도 가능하다. 이 경우 출력도 list.<br> 두 리스트의 순서는 중요하다: 출력 리스트의 요소 i는 두번째 인자의 리스트의 i번째 요소의 T.grad의 첫번째 인자의 gradient이다. 첫번째 인자는 스칼라이다.|

## Computing the Jacobian

 Jacorbian은 인풋에 대응하는 출력의 1계 편미분 도함수를 알 수 있다. 보통 야코비안은 야코비안 행렬식을 말하지만 theano에서는 야코비안 행렬을 뜻한다.
 Theano에서는 theano.gradient.jacobian()로 계산할 수 있다. 다음 글은 어떻게 수동으로 할 수 있는 지에 대한 글이다. 

수동으로 jacorbian을 계산하기 위해서 scan을 사용했다.

|Note|
|:--- |
|scan은 모든 재귀식의 문자를 허용하는 theano의 일반적인 operator이다. symbolic loop를 생성하고 최적화하는 과정이 어려울 때, scan으로 향상시킬 수 있다.|

In [2]:
import theano
import theano.tensor as T
x = T.dvector('x')
y = x ** 2
J, updates = theano.scan(lambda i, y,x : T.grad(y[i], x), sequences=T.arange(y.shape[0]), non_sequences=[y,x])
f = theano.function([x], J, updates=updates)
f([4, 4])

array([[ 8.,  0.],
       [ 0.,  8.]])

이 코드에서 우리는 T.arrange를 사용하여 0에서 y.shape[0]까지의 정수의 서열을 만들어냈다.
다음 우리는 이 서열의 루프를 도는 동안 각 단계에서 x에 대응하는 y[i]의 gradient를 계산했다.
scan은 자동적으로 야코비안 행렬에 대응되는 매트릭스의 모든 열을 연결시킨다.

|Note|
|:---|
|T.grad를 사용하는 경우 약간의 위험이 있다. 그 중 하나는  theano.scan(lambda y_i,x: T.grad(y_i,x), sequences=y, non_sequences=x)처럼 야코비 행렬을 다시 쓸 수 없다는 것이다. 그 이유는  $y[i]$가 여전히 존재해도 $y_i$는 어떤 경우에도 x에 대한 함수가 아니다.|

## Computing the Hessian

Theano에서 Hessian은 일반적인 수학적 용어와 같다: 이것은 scalar 출력과 vector input간의 함수의 2계 편미분 도함수를 생성한다.
Theano에서는 theano.gradient.hessian()로 계산할 수 있다. 다음 글은 어떻게 수동으로 할 수 있는 지에 대한 글이다.
 
야코비와 비슷하게 헤세도 계산할 수 있다. 한가지 다른 점은 cost가 스칼라일 경우 y에 대한 표현식에 대한 야코비를 계산하는 대신에 T.grad(cost,x)의 야코비를 계산했다.

In [10]:
x = T.dvector('x')
y = x ** 2
cost = y.sum()
gy = T.grad(cost, x)
H, updates = theano.scan(lambda i, gy,x : T.grad(gy[i], x), sequences=T.arange(gy.shape[0]), non_sequences=[gy, x])
f = theano.function([x], H, updates=updates)
f([4, 4])

array([[ 2.,  0.],
       [ 0.,  2.]])

## Jacobian times a Vector

종종 우리는 야코비와 백터 곱, 혹은 벡터와 야코비 곱의 알고리즘을 표현할 수 있다.야코비와 곱의 계산 결과를 비교하기 위해, 야코비의 실제 계산을 피하면서 원하는 결과를 계산하는 방법을 사용한다. 이것은 현저한 퍼포먼스 개선을 얻을 수 있다.
알고리즘에 대한 자세한 설명은 다음을 참고하라:

Barak A. Pearlmutter, “Fast Exact Multiplication by the Hessian”, Neural Computation, 1994

우리는 theano에 일반화하기 매우 어려운 최적화를 포함한 이런 패턴을 자동으로 구별하기를 원한다. 
그러므로 우리는 이 태스크를 수행할 특별한 함수를 제공한다.

### R-operator

R operator는 야코비안 행렬과 벡터의 곱을 계산하기 위해 $\frac{\partial f(x)}{\partial x} v$ 로 정의된다.
이 식은 x가 행렬이거나 텐서일 때 (야코비가 텐서가 되고 곱이 텐서 곱이 되는) 확장될 수 있다.
우리는 weight matrices 같은 각 표현식을 계산해야 하기 때문에 theano는 일반적인 양식을 지원한다.

R operation을 계산하기 위해서 v와 야코비안 행렬의 곱을 다음과 같이 구할 수 있다.

In [11]:
W = T.dmatrix('W')
V = T.dmatrix('V')
x = T.dvector('x')
y = T.dot(x, W)
JV = T.Rop(y, W, V)
f = theano.function([W, V, x], JV)
f([[1, 1], [1, 1]], [[2, 2], [2, 2]], [0,1])

array([ 2.,  2.])

### L-operator

R-operator와 비슷하게, L-operator는 a row vector와 야코비 행렬의 곱으로 계산된다. 수학식은 $v \frac{\partial
f(x)}{\partial x}$. L-operator 는 벡터 뿐 아니라 텐서를 지원한다. 아래와 같이 사용한다:

In [12]:
W = T.dmatrix('W')
v = T.dvector('v')
x = T.dvector('x')
y = T.dot(x, W)
VJ = T.Lop(y, W, v)
f = theano.function([v,x], VJ)
f([2, 2], [0, 1])

array([[ 0.,  0.],
       [ 2.,  2.]])

|note
|:---|
| v는 L-operator 와 the R-operator에서 다르다. L-operator는 출력과 동일한 shape의 point를 가지는 데 반해, R-operator는 입력 파라미터와 같은 shape의 point를 가진다. 더 나아가 두 연산의 결과도 다르다.L-operator의 결과는 input parameter와 같은 shape이지만, R-operator의 결과는 output의 shape과 같다.|

### Hessian times a Vector

헤세와 벡터의 곱을 계산해야 하는 경우
위에서 정의된 연산자를 사용하는 것이 실제 정확한 헤세를 구하고 곱을 계산하는 것보다 더 효율적이다.
헤세 행렬의 symmetry를 위해서, 같은 결과를 주지만 다른 퍼포먼스를 보여주는 두 옵션을 사용할 수 있다. 
그러므로 우리는 이 두 개의 방법을 사용하기 전에 프로파일링을 권한다:

In [13]:
x = T.dvector('x')
v = T.dvector('v')
y = T.sum(x ** 2)
gy = T.grad(y, x)
vH = T.grad(T.sum(gy * v), x)
f = theano.function([x, v], vH)
f([4, 4], [2, 2])

array([ 4.,  4.])

또는 R-operator를 사용하면

In [14]:
x = T.dvector('x')
v = T.dvector('v')
y = T.sum(x ** 2)
gy = T.grad(y, x)
Hv = T.Rop(gy, x, v)
f = theano.function([x, v], Hv)
f([4, 4], [2, 2])

array([ 4.,  4.])

# Final Pointers 

- `grad` 함수는 기호적으로 작동한다: Theano variables을 입력과 출력으로 한다.
- grad는 반복하며 적용될 수 있기 때문에 macro와 비교된다.
- Scalar costs는 grad로 직접적으로 다룬다. Arrays는 반복 적용을 통해 다룬다.
- Built-in 함수는 vector Jacobian 곱과 vector Hessian 곱을 효율적으로 계산한다.
- 전체 야코비 행렬과 헤세 행렬을 야코비와 벡터 곱을 계산하는 것처럼 효율적으로 계산하는 최적화 작업은 진행 중이다.